In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [28]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now("US/Eastern") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [29]:
pd.Timestamp.now(tz="US/Eastern").floor('h')

Timestamp('2025-03-02 21:00:00-0500', tz='US/Eastern')

In [36]:
df = fetch_hourly_rides(1)

2025-03-02 21:19:53,700 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 21:19:53,710 INFO: Initializing external client
2025-03-02 21:19:53,711 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 21:19:54,486 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.93s) 


In [37]:
df = df.sort_values(['pickup_hour']).reset_index(drop=True)

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1280 entries, 0 to 1279
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_hour         1280 non-null   datetime64[us, Etc/UTC]
 1   pickup_location_id  1280 non-null   int32                  
 2   rides               1280 non-null   int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 20.1 KB


In [39]:
df["pickup_hour"] = df["pickup_hour"].dt.tz_convert("US/Eastern")

In [40]:
df

,pickup_hour,pickup_location_id,rides
0,2025-03-02 15:00:00-05:00,22,0
1,2025-03-02 15:00:00-05:00,210,0
2,2025-03-02 15:00:00-05:00,261,58
3,2025-03-02 15:00:00-05:00,166,35
4,2025-03-02 15:00:00-05:00,184,0
...,...,...,...
1275,2025-03-02 19:00:00-05:00,34,0
1276,2025-03-02 19:00:00-05:00,211,44
1277,2025-03-02 19:00:00-05:00,58,0
1278,2025-03-02 19:00:00-05:00,208,0


In [41]:
df_pred = fetch_predictions(1)

2025-03-02 21:20:29,331 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 21:20:29,333 INFO: Initializing external client
2025-03-02 21:20:29,333 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 21:20:30,215 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.38s) 


In [42]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour
0,190,0.0,2025-03-02 21:00:00-05:00
1,18,0.0,2025-03-02 21:00:00-05:00
2,150,0.0,2025-03-02 21:00:00-05:00
3,113,1.0,2025-03-02 21:00:00-05:00
4,106,0.0,2025-03-02 21:00:00-05:00
...,...,...,...
251,15,0.0,2025-03-02 21:00:00-05:00
252,227,0.0,2025-03-02 21:00:00-05:00
253,64,0.0,2025-03-02 21:00:00-05:00
254,115,0.0,2025-03-02 21:00:00-05:00


In [ ]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [46]:
merged_df

,pickup_location_id,predicted_demand,pickup_hour,rides


In [58]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [59]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference
216,2025-02-22 23:00:00+00:00,2,0,0.0,0.0
381,2025-02-23 00:00:00+00:00,2,0,0.0,0.0
725,2025-02-23 02:00:00+00:00,2,0,0.0,0.0
784,2025-02-23 03:00:00+00:00,2,0,0.0,0.0
1139,2025-02-23 04:00:00+00:00,2,0,0.0,0.0
...,...,...,...,...,...
2729,2025-02-23 10:00:00+00:00,263,110,48.0,-62.0
2848,2025-02-23 11:00:00+00:00,263,130,62.0,-68.0
3244,2025-02-23 12:00:00+00:00,263,134,119.0,-15.0
3295,2025-02-23 13:00:00+00:00,263,154,133.0,-21.0


In [ ]:
import pandas as pd
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])

# Calculate the absolute error
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)

# Create a Plotly plot
fig = px.line(
    mae_by_hour,
    x='pickup_hour',
    y='MAE',
    title='Mean Absolute Error (MAE) by Pickup Hour',
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},
    markers=True
)

# Show the plot
fig.show()

In [61]:
mae_by_hour["MAE"].mean()

13.010889774236384